In [1]:
!pip install gym
import gym
import time
import numpy as np 

#Step3 Agent

* Agent是和环境environment交互的主体。

* predict()方法：输入观察值observation（或者说状态state），输出动作值

* sample()方法：在predict()方法基础上使用ε-greedy增加探索

* learn()方法：输入训练数据，完成一轮Q表格的更新

In [2]:
class QLearningAgent(object):
    def __init__(self, obs_n, act_n, learning_rate=0.01, gamma=0.9, e_greed=0.1):
        self.act_n = act_n      # 动作维度，有几个动作可选
        self.lr = learning_rate # 学习率
        self.gamma = gamma      # reward的衰减率
        self.epsilon = e_greed  # 按一定概率随机选动作
        self.Q = np.zeros((obs_n, act_n))

    # 根据输入观察值，采样输出的动作值，带探索
    def sample(self, obs):
        if np.random.uniform(0, 1) < (1.0 - self.epsilon): #根据table的Q值选动作
            action = self.predict(obs)
        else:
            action = np.random.choice(self.act_n) #有一定概率随机探索选取一个动作
        return action

    # 根据输入观察值，预测输出的动作值
    def predict(self, obs):
        Q_list = self.Q[obs, :]
        maxQ = np.max(Q_list)
        action_list = np.where(Q_list == maxQ)[0]  # maxQ可能对应多个action
        action = np.random.choice(action_list)
        return action

    # 学习方法，也就是更新Q-table的方法
    def learn(self, obs, action, reward, next_obs, done):
        """ off-policy
            obs: 交互前的obs, s_t
            action: 本次交互选择的action, a_t
            reward: 本次动作获得的奖励r
            next_obs: 本次交互后的obs, s_t+1
            done: episode是否结束
        """
        predict_Q = self.Q[obs, action]
        if done:
            target_Q = reward # 没有下一个状态了
        else:
            target_Q = reward + self.gamma * np.max(self.Q[next_obs, :]) # Q-learning
        self.Q[obs, action] += self.lr * (target_Q - predict_Q) # 修正q

    # 把 Q表格 的数据保存到文件中
    def save(self):
        npy_file = './q_table.npy'
        np.save(npy_file, self.Q)
        print(npy_file + ' saved.')
    
    # 从文件中读取数据到 Q表格
    def restore(self, npy_file='./q_table.npy'):
        self.Q = np.load(npy_file)
        print(npy_file + ' loaded.')

#Step4 Training && Test（训练&&测试）

* run_episode()：agent在一个episode中训练的过程，使用agent.sample()与环境交互，使用agent.learn()训练Q表格。
* test_episode()：agent在一个episode中测试效果，评估目前的agent能在一个episode中拿到多少总reward。

In [3]:
# train.py

def run_episode(env, agent, render=False):
    total_steps = 0 # 记录每个episode走了多少step
    total_reward = 0

    obs = env.reset() # 重置环境, 重新开一局（即开始新的一个episode）

    while True:
        action = agent.sample(obs) # 根据算法选择一个动作
        next_obs, reward, done, _ = env.step(action) # 与环境进行一个交互
        # 训练 Q-learning算法
        agent.learn(obs, action, reward, next_obs, done)

        obs = next_obs  # 存储上一个观察值
        total_reward += reward
        total_steps += 1 # 计算step数
        if render:
            env.render() #渲染新的一帧图形
        if done:
            break
    return total_reward, total_steps

def test_episode(env, agent):
    total_reward = 0
    obs = env.reset()
    while True:
        action = agent.predict(obs) # greedy
        next_obs, reward, done, _ = env.step(action)
        total_reward += reward
        obs = next_obs
        # time.sleep(0.5)
        # env.render()
        if done:
            break
    return total_reward

#Step5 创建环境和Agent，启动训练

In [4]:
# 使用gym创建悬崖环境
env = gym.make("CliffWalking-v0")  # 0 up, 1 right, 2 down, 3 left

# 创建一个agent实例，输入超参数
agent = QLearningAgent(
    obs_n=env.observation_space.n,
    act_n=env.action_space.n,
    learning_rate=0.1,
    gamma=0.9,
    e_greed=0.1)


# 训练500个episode，打印每个episode的分数
for episode in range(500):
    ep_reward, ep_steps = run_episode(env, agent, False)
    print('Episode %s: steps = %s , reward = %.1f' % (episode, ep_steps, ep_reward))

# 全部训练结束，查看算法效果
test_reward = test_episode(env, agent)
print('test reward = %.1f' % (test_reward))

Episode 0: steps = 479 , reward = -1568.0
Episode 1: steps = 544 , reward = -940.0
Episode 2: steps = 219 , reward = -318.0
Episode 3: steps = 424 , reward = -721.0
Episode 4: steps = 237 , reward = -435.0
Episode 5: steps = 85 , reward = -85.0
Episode 6: steps = 218 , reward = -317.0
Episode 7: steps = 183 , reward = -579.0
Episode 8: steps = 167 , reward = -167.0
Episode 9: steps = 138 , reward = -237.0
Episode 10: steps = 77 , reward = -77.0
Episode 11: steps = 133 , reward = -232.0
Episode 12: steps = 372 , reward = -966.0
Episode 13: steps = 68 , reward = -68.0
Episode 14: steps = 85 , reward = -85.0
Episode 15: steps = 118 , reward = -217.0
Episode 16: steps = 45 , reward = -45.0
Episode 17: steps = 139 , reward = -139.0
Episode 18: steps = 131 , reward = -230.0
Episode 19: steps = 42 , reward = -42.0
Episode 20: steps = 78 , reward = -78.0
Episode 21: steps = 119 , reward = -119.0
Episode 22: steps = 77 , reward = -77.0
Episode 23: steps = 72 , reward = -72.0
Episode 24: steps =